<a href="https://colab.research.google.com/github/DmytroBuzanov/DS_ML/blob/main/wifi_localization_Training_model_6_5_routers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 1.x
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.callbacks import EarlyStopping 
import time

from sklearn.preprocessing import StandardScaler
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD  
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

from google.colab import drive  
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('.../wifi_localization.txt', names=['WS1',	'WS2',	'WS3',	'WS4',	'WS5',	'WS6',	'WS7',	'Class'], sep='\t')

df.drop('WS6', axis=1, inplace=True)
df.drop('WS7', axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df.values[:, :5],    
                                                    df.values[:, -1] - 1, 
                                                    test_size=0.25, 
                                                    random_state=20)
X_train, X_val, y_train, y_val = train_test_split(X_train,                         
                                                y_train, 
                                                test_size=0.10, 
                                                random_state=20)

In [5]:
scaler = StandardScaler()
print(scaler.fit(X_train))

X_train_scaled = scaler.transform(X_train)                        
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)


def create_model(learn_rate=0.01, 
                 momentum=0.8,
                 k_l1=0.01, 
                 k_l2=0, 
                 k_l1_l2=0,
                 neurons_1=6,
                 neurons_2=9):
  model = Sequential([
    Dense(neurons_1, input_shape=(5,), activation='relu',kernel_regularizer=regularizers.l1(k_l1)),
    Dense(neurons_2, activation='relu',kernel_regularizer=regularizers.l1(k_l1)),
    Dense(4, activation='softmax',kernel_regularizer=regularizers.l1(k_l1))])

  model.compile(optimizer=SGD(lr=learn_rate, momentum=momentum),              
                loss='sparse_categorical_crossentropy', 
                metrics=['sparse_categorical_accuracy'])
                              
  return model

early_Stop = EarlyStopping(monitor='val_loss',                           
                           patience=20,
                           verbose=2,                        
                           restore_best_weights=True)


StandardScaler(copy=True, with_mean=True, with_std=True)


In [6]:
model = create_model(k_l1=0.01, learn_rate=0.01, momentum=0.8, neurons_1=6, neurons_2=9)
model.fit(X_train_scaled, y_train, epochs=1000, callbacks = [early_Stop], validation_data=(X_val_scaled, y_val), verbose=0)

print('test: ', model.evaluate(X_test_scaled, y_test))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Restoring model weights from the end of the best epoch
Epoch 00780: early stopping
500/500 [==============================] - 0s 50us/sample - loss: 0.2705 - sparse_categorical_accuracy: 0.9820
test:  [0.2705286670923233, 0.982]
